<a href="https://colab.research.google.com/github/Rishabh5752/Text_Attack_Exp/blob/main/BERT_base_uncased_Rotten_Tomatoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install textattack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 93.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 92.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.

In [2]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 41.7 MB/s eta 0:00:00


In [3]:
import textattack
import transformers

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-rotten_tomatoes")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-rotten_tomatoes")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

In [5]:
# Construct our four components for `Attack`

from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.transformations import WordSwap

In [6]:
goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)

constraints = [

    RepeatModification(),
    StopwordModification(),
    WordEmbeddingDistance(min_cos_sim=0.7)

]
from textattack.search_methods import GreedySearch

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:10<00:00, 46.3MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpqo298epw.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [7]:
from textattack.search_methods import GreedySearch

In [8]:
transformation = textattack.transformations.word_swaps.word_swap_embedding.WordSwapEmbedding()
search_method = textattack.search_methods.GreedySearch()

In [9]:
#attack = textattack.attack_recipes.deepwordbug_gao_2018.DeepWordBugGao2018.build(model_wrapper)
attack =  textattack.Attack(goal_function, constraints, transformation, search_method)

#datasets.load_dataset('emotion', split =['test'])
#dataset=datasets.load_dataset('sms_spam', split =['test'])
dataset = textattack.datasets.HuggingFaceDataset("rotten_tomatoes", dataset_columns=(["text"],'label'))
#attack =  textattack.Attack(goal_function, constraints, transformation, search_method)

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

textattack: Loading datasets dataset rotten_tomatoes, split train.


In [10]:
attack_args = textattack.AttackArgs(

    num_examples=100,
    disable_stdout=True

)
attacker = textattack.Attacker(attack, dataset, attack_args)

attacker.attack_dataset()

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 80 / 20 / 0 / 100: 100%|██████████| 100/100 [13:23<00:00,  8.03s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 80     |
| Number of failed attacks:     | 20     |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 80.0%  |
| Average perturbed word %:     | 15.03% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 244.42 |
+-------------------------------+--------+


In [11]:
from textattack import Attacker

attacker = Attacker(attack, dataset)
attack_results=attacker.attack_dataset()

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



 10%|█         | 1/10 [00:02<00:26,  2.97s/it]

--------------------------------------------- Result 1 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:03<00:29,  3.31s/it]

[[Positive (100%)]] --> [[Negative (99%)]]

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash [[even]] greater than arnold schwarzenegger , jean-claud van damme or steven segal .

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash [[too]] greater than arnold schwarzenegger , jean-claud van damme or steven segal .




...

--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (87%)]]

perhaps no picture ever made has more literally showed that the road to [[hell]] is paved with [[good]] intentions .

perhaps no picture ever made has more literally showed that the road to [[shit]] is paved with [[guten]] intentions .




[Succeeded / Failed / Skipped / Total] 7 / 2 / 0 / 9:  90%|█████████ | 9/10 [00:54<00:06,  6.07s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (99%)]]

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow [[pulls]] it off .

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow [[withdrew]] it off .




[Succeeded / Failed / Skipped / Total] 7 / 3 / 0 / 10: 100%|██████████| 10/10 [01:05<00:00,  6.54s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

take care of my cat offers a refreshingly different slice of asian cinema .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 30.0%  |
| Attack success rate:          | 70.0%  |
| Average perturbed word %:     | 7.76%  |
| Average num. words per input: | 19.5   |
| Avg num queries:              | 197.6  |
+-------------------------------+--------+
